# HATE SPEECH DETECTION

## DATA PREPROCESSING 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [4]:
path = '/home/spriyanshu723/prince/detectinsult/'
!ls {path}

detecting-insults-in-social-commentary.zip  test.csv
impermium_verification_labels.csv	    test_with_solutions.csv
impermium_verification_set.csv		    train.csv
sample_submission_null.csv


In [5]:
data = pd.read_csv('/home/spriyanshu723/prince/detectinsult/train.csv')

In [6]:
data.head()

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [7]:
data = data.drop(['Date'], axis=1)
data = data[['Comment' , 'Insult']]

In [8]:
data.head()

,Comment,Insult
0,"""You fuck your dad.""",1
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3947 entries, 0 to 3946
Data columns (total 2 columns):
Comment    3947 non-null object
Insult     3947 non-null int64
dtypes: int64(1), object(1)
memory usage: 61.8+ KB


In [10]:
data.describe()

,Insult
count,3947.000000
mean,0.265771
std,0.441799
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [11]:
data.shape

(3947, 2)

## RETRIVING TOP WORDS FROM HATE DATASET 

In [12]:
data.head()

,Comment,Insult
0,"""You fuck your dad.""",1
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0


In [13]:
data_hate_four = data.loc[data['Insult'] == 1]
data_hate_four.head()

,Comment,Insult
0,"""You fuck your dad.""",1
7,"""shut the fuck up. you and the rest of your fa...",1
8,"""Either you are fake or extremely stupid...may...",1
9,"""That you are an idiot who understands neither...",1
15,"""FOR SOME REASON U SOUND RETARDED. LOL. DAMN. ...",1


In [14]:
data_hate_four.shape

(1049, 2)

In [15]:
data_unhate_four = data.loc[data['Insult'] == 0]
data_unhate_four.head()

,Comment,Insult
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0
5,"""@SDL OK, but I would hope they'd sign him to ...",0


In [16]:
data_unhate_four.shape

(2898, 2)

In [17]:
2898 + 1049

3947

In [18]:
data_hate_four.head()

,Comment,Insult
0,"""You fuck your dad.""",1
7,"""shut the fuck up. you and the rest of your fa...",1
8,"""Either you are fake or extremely stupid...may...",1
9,"""That you are an idiot who understands neither...",1
15,"""FOR SOME REASON U SOUND RETARDED. LOL. DAMN. ...",1


In [19]:
data_hate_four = data_hate_four.drop(['Insult'], axis=1)
data_hate_four['class'] = 1
data_hate_four.head()

,Comment,class
0,"""You fuck your dad.""",1
7,"""shut the fuck up. you and the rest of your fa...",1
8,"""Either you are fake or extremely stupid...may...",1
9,"""That you are an idiot who understands neither...",1
15,"""FOR SOME REASON U SOUND RETARDED. LOL. DAMN. ...",1


In [20]:
data_unhate_four = data_unhate_four.drop(['Insult'], axis=1)
data_unhate_four['class'] = 0
data_unhate_four.head()

,Comment,class
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0
5,"""@SDL OK, but I would hope they'd sign him to ...",0


In [21]:
data_hate_four.shape

(1049, 2)

In [22]:
data_unhate_four.shape

(2898, 2)

In [23]:
with open('a_hate_four', 'wb') as fp:
    pickle.dump(data_hate_four, fp)

In [24]:
with open('a_unhate_four', 'wb') as fp:
    pickle.dump(data_unhate_four, fp)

In [25]:
hate_df = data_hate_four['Comment']
hate_df.head()

0                                  "You fuck your dad."
7     "shut the fuck up. you and the rest of your fa...
8     "Either you are fake or extremely stupid...may...
9     "That you are an idiot who understands neither...
15    "FOR SOME REASON U SOUND RETARDED. LOL. DAMN. ...
Name: Comment, dtype: object

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
import nltk
from nltk.tokenize import TweetTokenizer 
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

In [28]:
one = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [29]:
one

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f9daa54e9d8>, use_idf=1,
        vocabulary=None)

In [30]:
trn_term_doc = one.fit_transform(hate_df)

In [31]:
trn_term_doc

<1049x3402 sparse matrix of type '<class 'numpy.float64'>'
	with 37231 stored elements in Compressed Sparse Row format>

In [34]:
indices_hate = np.argsort(one.idf_)[::-1]
features_hate = one.get_feature_names()
top_n = 100
top_features_hate = [features_hate[i] for i in indices_hate[:top_n]]
print(top_features_hate)

['yup', 'if he', 'i never', 'i suppose', 'i swear i', 'i wanna', 'idi0t', 'idiot . \\xa0', 'idiot and', 'idiotic comment of', 'idiots like', 'idiots like you', 'idiots posting', 'if a', 'ignorance . \\nplease', 'i guess', 'ignorant and', 'ignore', 'ill', 'ima', 'imbecile', 'immediately', 'in ,', 'in america', 'in hell', 'in her', 'in order to', 'in the fat', 'i mean', 'i do', 'in your head', 'how much', 'hitting', 'hmm', 'ho', 'hole you', 'holy', 'homo', 'hope you get', 'horrible', 'how can', 'how can you', 'how dare', 'how many times', 'how stupid', "i can '", 'huge', 'hugging', 'huh', 'huh ?', 'human being .', 'humans', 'hunt', 'i .', 'i :', 'i : t', 'i >', 'i am a', 'in the first', 'in your mouth', 'himself', 'know nothing', 'just dumb', 'just keep', 'just plain', 'just shut', 'keep it', 'keep your', 'kept', 'kill the', 'kill you', 'king', 'knees', 'knew', 'knowing', 'joking', 'l :', 'l : l', 'l : s', 'lady', 'land', 'language', 'law', 'lazy', 'leg', 'liar', 'lick', 'life . "', 'jus

### SAVING THE DATA FROM HATE DATASET 

In [35]:
indices_hate = np.argsort(one.idf_)[::-1]
features_hate = one.get_feature_names()
top_n = 5000
top_features_hate = [features_hate[i] for i in indices_hate[:top_n]]

In [36]:
with open('z_hate_four', 'wb') as fp:
    pickle.dump(top_features_hate, fp)

## RETRIVING TOP WORDS FROM UNHATE DATASET 

In [37]:
data_unhate_four.head()

,Comment,class
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0
5,"""@SDL OK, but I would hope they'd sign him to ...",0


In [39]:
unhate_df = data_unhate_four['Comment']
unhate_df.head()

1    "i really don't understand your point.\xa0 It ...
2    "A\\xc2\\xa0majority of Canadians can and has ...
3    "listen if you dont wanna get married to a man...
4    "C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...
5    "@SDL OK, but I would hope they'd sign him to ...
Name: Comment, dtype: object

In [40]:
trn_term_doc = one.fit_transform(unhate_df)

In [41]:
trn_term_doc

<2898x11645 sparse matrix of type '<class 'numpy.float64'>'
	with 141299 stored elements in Compressed Sparse Row format>

In [42]:
indices_unhate = np.argsort(one.idf_)[::-1]
features_unhate = one.get_feature_names()
top_n = 100
top_features_unhate = [features_unhate[i] for i in indices_unhate[:top_n]]
print(top_features_unhate)

['i just want', 'billy', '@ blaine', 'seriously ?', '@ blaine jungwirth', 'serious ?', '@ dan', 'series of', '@ dan _', 'sentence .', '@ david @', '@ ikathyv', 'sensitive', 'bin laden', 'sense that', 'is an idiot', 'is best', 'serving', 'send a', '@ jasonhairydawg', '@ kick', 'is broken', '@ kick ass', 'is coming', 'is definitely', 'is different', '@ notborneverymin', 'is either', '@ yinzerinct\\xa0\\xa0', 'is far', '@ yinzerinct\\xa0\\xa0 @', 'billions', "@ ' micah", "@ '", 'sheeple', 'bit . ly', 'ireland', '? it is', 'she should', 'irish', 'she knows', 'she is a', '? lol', 'irony', 'irrational', 'bitch .', 'bitch "', 'bit more', 'share .', 'is . \\xa0', 'is a big', 'set .', 'is a bit', 'is a good', '? yes', 'sexually', 'is a joke', 'sexist', 'sex with', 'sex -', 'sex ,', '? you are', '? your', 'seven', 'sets', 'is a very', 'is for the', '\\\\n " the', 'is full of', 'school .', 'better or', 'scum like', 'is our', 'is out', 'better off', 'screen', 'is part', 'scream', 'is part of', 'is

### SAVING TOP WORDS FROM UNHATE DATASET 

In [43]:
indices_unhate = np.argsort(one.idf_)[::-1]
features_unhate = one.get_feature_names()
top_n = 5000
top_features_unhate = [features_unhate[i] for i in indices_unhate[:top_n]]

In [44]:
with open('z_unhate_four', 'wb') as fp:
    pickle.dump(top_features_unhate, fp)